# Using `pyologger` data processing pipeline with `DiveDB`
Uses classes `Metadata` and `DataReader` to facilitate data intake, processing, and alignment. 

## Read deployment metadata

In [ ]:
# Import libraries and set working directory (adjust to fit your preferences)
import os
import pickle
from pyologger.load_data.datareader import DataReader
from pyologger.load_data.metadata import Metadata
from pyologger.plot_data.plotter import plot_tag_data_interactive, plot_tag_data, load_color_mapping, save_color_mapping, plot_depth_correction
from pyologger.calibrate_data.calibrate_acc_mag import *


# Change the current working directory to the root directory
# os.chdir("/Users/fbar/Documents/GitHub/pyologger")
os.chdir("/Users/jessiekb/Documents/GitHub/pyologger")

root_dir = os.getcwd()
data_dir = os.path.join(root_dir, "data")

# Verify the current working directory
print(f"Current working directory: {root_dir}")

In [ ]:
import xarray as xr
#import netcdf4

# Load the NetCDF file
file_path = f"{data_dir}/2004001_TrackTDR_RawCurated.nc"
dataset = xr.open_dataset(file_path)

# Print the dataset information
print(dataset)

In [ ]:
# Initialize the info class
metadata = Metadata()
metadata.fetch_databases(verbose=False)

# Save databases
dep_db = metadata.get_metadata("dep_DB")
logger_db = metadata.get_metadata("logger_DB")
rec_db = metadata.get_metadata("rec_DB")
animal_db = metadata.get_metadata("animal_DB")

In [ ]:
# Check out the deployment dataframe.
dep_db_sorted = dep_db.sort_values('Rec Date')
dep_db_sorted

## Read files in deployment folder

Steps for Processing Deployment Data:

1. **Select Deployment Folder**:
   - **Description:** Asks the user for input to select a deployment folder to kick off the data reading process. In your folder name, you can have any suffix after Deployment ID. It will check and stop if there are two that fit.
   - **Function Used:** `check_deployment_folder()`

2. **Initialize Deployment Folder**:
   - **Description:** Starts the main `read_files` process with the selected deployment folder.
   - **Function Used:** `read_files()`

3. **Fetch Metadata**:
   - **Description:** Retrieve necessary data from the metadata database, including logger information.
   - **Function Used:** `metadata.fetch_databases()`

4. **Organize Files by Logger ID**:
   - **Description:** Group files by logger ID for processing.
   - **Function Used:** `read_files()` (This is the main function)

5. **Check for Existing Processed Files**:
   - **Description:** Verify if the outputs folder already contains processed files for each logger. Skip reprocessing if all necessary files are present.
   - **Function Used:** `check_outputs_folder()`

6. **Process UBE Files**:
   - **Description:** For each UFI logger with UBE files, process and save the data.
   - **Function Used:** `process_ube_file()`

7. **Process CSV Files**:
   - **Description:** For each logger with multiple CSV files, concatenate them, and save the combined data.
   - **Function Used:** `concatenate_and_save_csvs()`

8. **Final Outputs**:
   - **Description:** Ensure all processed data is saved in the outputs folder with appropriate filenames.
   - **Functions Used:** `save_data()`

In [ ]:
# Assuming you have the metadata and dep_db loaded:
datareader = DataReader()
deployment_folder = datareader.check_deployment_folder(dep_db, data_dir)

if deployment_folder:
    datareader.read_files(metadata, save_csv=True, save_parq=True)

In [ ]:
# Optionally look at first notes that have been read in
#datareader.selected_deployment['Time Zone']
#datareader.info['UF-01']
datareader.notes_df[0:5]
#datareader.data['CC-96']
#datareader.data['UF-01']
#datareader.metadata['channelnames']

## Inspect data

In [ ]:
# Load the data_reader object from the pickle file
pkl_path = os.path.join(deployment_folder, 'outputs', 'data.pkl')

with open(pkl_path, 'rb') as file:
    data_pkl = pickle.load(file)

for logger_id, info in data_pkl.info.items():
    sampling_frequency = info.get('datetime_metadata', {}).get('fs', None)
    if sampling_frequency is not None:
        # Format the sampling frequency to 5 significant digits
        print(f"Sampling frequency for {logger_id}: {sampling_frequency} Hz")
    else:
        print(f"No sampling frequency available for {logger_id}")

data_pkl.info['CC-96']['datetime_metadata']['fs']

### Plot data
Make an interactive plot

In [ ]:
# Load color mappings
color_mapping_path = os.path.join(root_dir, 'color_mappings.json')

# Streamlit sidebar for time range selection
imu_logger_to_use = 'CC-96'
ephys_logger_to_use = 'UF-01'

imu_df = data_pkl.data[imu_logger_to_use]
ephys_df = data_pkl.data[ephys_logger_to_use]
overlap_start_time = max(imu_df['datetime'].min(), ephys_df['datetime'].min()).to_pydatetime()
overlap_end_time = min(imu_df['datetime'].max(), ephys_df['datetime'].max()).to_pydatetime()

# Define notes to plot
notes_to_plot = {
    'heartbeat_manual_ok': 'ecg',
    'exhalation_breath': 'depth'
}

# Plotting
fig = plot_tag_data_interactive(data_pkl, imu_channels=['depth', 'corrdepth', 'accX', 'accY', 'accZ', 'gyrX', 'gyrY', 'gyrZ', 'magX', 'magY', 'magZ'], 
                                ephys_channels=['ecg'], 
                                imu_logger=imu_logger_to_use, 
                                ephys_logger=ephys_logger_to_use, 
                                time_range=(overlap_start_time, overlap_end_time), 
                                note_annotations=notes_to_plot, 
                                color_mapping_path=color_mapping_path)

fig.show()

## Find dives
Involves a zero offset correction with `zoc`

In [ ]:
from pyologger.process_data.sampling import upsample
from pyologger.calibrate_data.zoc import *
from pyologger.plot_data.plotter import plot_depth_correction

# Load the depth and temperature data
depth_data = data_pkl.data['CC-96']['depth'].values
temp_data = data_pkl.data['CC-96'].get('tempIMU')
sampling_rate = int(data_pkl.info['CC-96']['datetime_metadata']['fs'])
datetime_data = pd.to_datetime(data_pkl.data['CC-96']['datetime'])

# Sidebar for parameters
threshold = 0.1 # meters
min_duration = 30 # seconds
depth_threshold = 5 # meters
apply_temp_correction = False

# Dive detection parameters
min_depth_threshold = 1.0
dive_duration_threshold = 10
smoothing_window = value=5

# Process depth data
first_derivative, downsampled_depth = smooth_downsample_derivative(depth_data, original_sampling_rate=sampling_rate, downsampled_sampling_rate=1)

# Detect flat chunks (potential surface intervals)
flat_chunks = detect_flat_chunks(
    depth=downsampled_depth, 
    datetime_data=datetime_data[::int(sampling_rate)],  # Adjust datetime data to match downsampled depth
    first_derivative=first_derivative, 
    threshold=threshold, 
    min_duration=min_duration, 
    depth_threshold=depth_threshold, 
    original_sampling_rate=400, 
    downsampled_sampling_rate=1
)
num_flat_chunks = len(flat_chunks)
print(f"Number of potential surface intervals detected: {num_flat_chunks}")

# Apply zero offset correction
corrected_depth_temp, corrected_depth_no_temp, depth_correction = apply_zero_offset_correction(
    depth=downsampled_depth, 
    temp=temp_data.values if temp_data is not None else None, 
    flat_chunks=flat_chunks
)

# Upsample and adjust the corrected depths to match original sampling rate
upsampling_factor = int(sampling_rate / 1)
repeated_corrected_depth_temp = upsample(corrected_depth_temp, upsampling_factor, len(depth_data))
repeated_corrected_depth_no_temp = upsample(corrected_depth_no_temp, upsampling_factor, len(depth_data))

# Detect dives in the corrected depth data
dives = find_dives(
    depth_series=repeated_corrected_depth_no_temp,
    datetime_data=datetime_data,
    min_depth_threshold=min_depth_threshold,
    sampling_rate=sampling_rate,
    duration_threshold=dive_duration_threshold,
    smoothing_window=smoothing_window
)
num_dives = len(dives)
print(f"Number of dives detected: {num_dives}")


In [ ]:
?plot_depth_correction

### Plot data

In [ ]:
# Plotting
dec_factor = int(upsampling_factor)

fig = plot_depth_correction(datetime_data, dec_factor, depth_data, first_derivative, 
                            repeated_corrected_depth_temp, repeated_corrected_depth_no_temp, 
                            depth_correction, dives, flat_chunks, temp_data, apply_temp_correction)
fig.show()


### Save data

In [ ]:
# Save the corrected depth back to the data structure
if apply_temp_correction:
    data_pkl.data['CC-96']['corrdepth'] = repeated_corrected_depth_temp
else:
    data_pkl.data['CC-96']['corrdepth'] = repeated_corrected_depth_no_temp

with open(pkl_path, 'wb') as file:
        pickle.dump(data_pkl, file)

## Calibrate ACC & MAG

In [ ]:
# Assuming `data_pkl` is already loaded and contains your data
accX = data_pkl.data['CC-96']['accX'].values
accY = data_pkl.data['CC-96']['accY'].values
accZ = data_pkl.data['CC-96']['accZ'].values
magX = data_pkl.data['CC-96']['magX'].values
magY = data_pkl.data['CC-96']['magY'].values
magZ = data_pkl.data['CC-96']['magZ'].values

# Combine the accelerometer and magnetometer data into nx3 matrices
acc_data = np.vstack((accX, accY, accZ)).T
mag_data = np.vstack((magX, magY, magZ)).T

# Get the sampling rate from the data structure
sampling_rate = int(data_pkl.info['CC-96']['datetime_metadata']['fs'])

# Call the check_AM function
AMcheck = compute_field_intensity_and_inclination(acc_data, mag_data, sampling_rate)

# Access the field intensity and inclination angle
field_intensity_acc = AMcheck['field_intensity'][:, 0]  # Field intensity of accelerometer data
field_intensity_mag = AMcheck['field_intensity'][:, 1]  # Field intensity of magnetometer data
inclination_angle = AMcheck['inclination_angle']

# Print results
print("Field Intensity:\n", field_intensity_acc)
print("Field Intensity:\n", field_intensity_mag)
print("Inclination Angle (degrees):\n", inclination_angle)

In [ ]:
# Save new fields to data_pkl.data['CC-96']
data_pkl.data['CC-96']['field_intensity_acc'] = field_intensity_acc
data_pkl.data['CC-96']['field_intensity_mag'] = field_intensity_mag
data_pkl.data['CC-96']['inclination_angle'] = inclination_angle

# Prepare channels to plot
imu_channels_to_plot = ['accX', 'accY', 'accZ', 'field_intensity_acc', 'field_intensity_mag', 'inclination_angle']
ephys_channels_to_plot = []
imu_logger_to_use = 'CC-96'
ephys_logger_to_use = 'UF-01'

# Get the overlapping time range
imu_df = data_pkl.data[imu_logger_to_use]
ephys_df = data_pkl.data[ephys_logger_to_use]
start_time = max(imu_df['datetime'].min(), ephys_df['datetime'].min()).to_pydatetime()
end_time = min(imu_df['datetime'].max(), ephys_df['datetime'].max()).to_pydatetime()

plot_tag_data_interactive(data_pkl, imu_channels_to_plot, ephys_channels=ephys_channels_to_plot, 
                          imu_logger=imu_logger_to_use, ephys_logger=ephys_logger_to_use, 
                          time_range=(start_time, end_time), color_mapping_path=color_mapping_path)

### Apply adjustments

In [ ]:
# Apply the fix_offset_3d function to adjust accelerometer data
result = estimate_offset_triaxial(acc_data)

# Extract the adjusted data and calibration info
adjusted_data_acc = result['X']
calibration_info_acc = result['G']

print("Adjusted Data:\n", adjusted_data_acc)
print("Calibration Info:\n", calibration_info_acc)

data_pkl.data['CC-96']['accX_adjusted'] = adjusted_data_acc[:, 0]
data_pkl.data['CC-96']['accY_adjusted'] = adjusted_data_acc[:, 1]
data_pkl.data['CC-96']['accZ_adjusted'] = adjusted_data_acc[:, 2]
data_pkl.info['CC-96']['calibration_info'] = {}
data_pkl.info['CC-96']['calibration_info'] = calibration_info_acc

# Apply the fix_offset_3d function to adjust magnetometer data
result = estimate_offset_triaxial(mag_data)

# Extract the adjusted data and calibration info
adjusted_data_mag = result['X']
calibration_info_mag = result['G']

print("Adjusted Data:\n", adjusted_data_mag)
print("Calibration Info:\n", calibration_info_mag)

data_pkl.data['CC-96']['magX_adjusted'] = adjusted_data_mag[:, 0]
data_pkl.data['CC-96']['magY_adjusted'] = adjusted_data_mag[:, 1]
data_pkl.data['CC-96']['magZ_adjusted'] = adjusted_data_mag[:, 2]
data_pkl.info['CC-96']['calibration_info'] = {}
data_pkl.info['CC-96']['calibration_info'] = calibration_info_mag

### Re-run check AM. 
Re-run the field intensity and inclination angle calculations to see if now the values are closer to expected.

In [ ]:
# Assuming `data_pkl` is already loaded and contains your data
accX = data_pkl.data['CC-96']['accX_adjusted'].values
accY = data_pkl.data['CC-96']['accY_adjusted'].values
accZ = data_pkl.data['CC-96']['accZ_adjusted'].values
magX = data_pkl.data['CC-96']['magX_adjusted'].values
magY = data_pkl.data['CC-96']['magY_adjusted'].values
magZ = data_pkl.data['CC-96']['magZ_adjusted'].values

# Combine the accelerometer and magnetometer data into nx3 matrices
acc_data = np.vstack((accX, accY, accZ)).T
mag_data = np.vstack((magX, magY, magZ)).T

# Get the sampling rate from the data structure
sampling_rate = int(data_pkl.info['CC-96']['datetime_metadata']['fs'])

# Call the check_AM function
AMcheck2 = compute_field_intensity_and_inclination(acc_data, mag_data, sampling_rate)

# Access the field intensity and inclination angle
field_intensity_acc2 = AMcheck2['field_intensity'][:, 0]  # Field intensity of accelerometer data
field_intensity_mag2 = AMcheck2['field_intensity'][:, 1]  # Field intensity of magnetometer data
inclination_angle2 = AMcheck2['inclination_angle']

# Print results
print("Field Intensity:\n", field_intensity_acc2)
print("Field Intensity:\n", field_intensity_mag2)
print("Inclination Angle (degrees):\n", inclination_angle2)

# Save new fields to data_pkl.data['CC-96']
data_pkl.data['CC-96']['field_intensity_acc2'] = field_intensity_acc2
data_pkl.data['CC-96']['field_intensity_mag2'] = field_intensity_mag2
data_pkl.data['CC-96']['inclination_angle2'] = inclination_angle2

# Example usage in Streamlit
imu_channels_to_plot = ['accX', 'accY', 'accZ', 'field_intensity_acc2', 'field_intensity_mag2', 'inclination_angle2']
ephys_channels_to_plot = []
imu_logger_to_use = 'CC-96'
ephys_logger_to_use = 'UF-01'

# Get the overlapping time range
imu_df = data_pkl.data[imu_logger_to_use]
ephys_df = data_pkl.data[ephys_logger_to_use]
start_time = max(imu_df['datetime'].min(), ephys_df['datetime'].min()).to_pydatetime()
end_time = min(imu_df['datetime'].max(), ephys_df['datetime'].max()).to_pydatetime()

plot_tag_data_interactive(data_pkl, imu_channels_to_plot, ephys_channels=ephys_channels_to_plot, 
                          imu_logger=imu_logger_to_use, ephys_logger=ephys_logger_to_use, 
                          time_range=(start_time, end_time), color_mapping_path=color_mapping_path)



### Fix inclination angle
Here is where you would fix the axes of triaxial sensor data (if the sensor axis differs from the tag axis)

## Tag to animal frame
### Start with an orientation table

In [ ]:
# OTAB Matrix Example
OTAB = np.array([
    [0, 0, 0.0, 0.0, 0.0]       # Initial orientation at time 0
])

In [ ]:
from pyologger.calibrate_data.prh_predictor import *

?prh_predictor2

In [ ]:
sampling_rate

In [ ]:
data_pkl.notes_df

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

def plot_acc_for_exhalation_breaths(data_pkl):
    """
    Plot accX, accY, and accZ values for each exhalation breath event and 
    return the average acceleration vector (abar0) around the events.

    Parameters
    ----------
    data_pkl : object
        The structured data object containing sensor data and notes_df.

    Returns
    -------
    abar0 : numpy.ndarray
        A vector containing the mean of accX, accY, and accZ during the 10 seconds 
        surrounding each exhalation breath event.
    """
    # Extract the relevant accelerometer data
    accX = data_pkl.data['CC-96']['accX'].values
    accY = data_pkl.data['CC-96']['accY'].values
    accZ = data_pkl.data['CC-96']['accZ'].values
    datetime_data = data_pkl.data['CC-96']['datetime'].values

    # Convert datetime_data to numpy.datetime64 for proper subtraction
    datetime_data = np.array(datetime_data, dtype='datetime64[ns]')

    # Filter the notes_df for 'exhalation_breath' events
    breath_events = data_pkl.notes_df[data_pkl.notes_df['key'] == 'exhalation_breath']

    # Initialize lists to store the surrounding data
    accX_segments = []
    accY_segments = []
    accZ_segments = []

    # Plot the accelerometer data for each breath event
    plt.figure(figsize=(15, 10))

    for _, event in breath_events.iterrows():
        event_time = event['datetime']  # Convert event_time to numpy.datetime64

        # Round to the nearest second
        event_time_rounded = event_time.round('1s')

        # Find the index of the rounded event time
        rounded_time_index = datetime_data.tolist().index(event_time_rounded)

        # Define the window of ±5 seconds around the breath event
        time_window = int(5 * data_pkl.info['CC-96']['datetime_metadata']['fs'])
        start_index = max(rounded_time_index - time_window, 0)
        end_index = min(rounded_time_index + time_window, len(datetime_data))

        # Extract the segments
        accX_segment = accX[start_index:end_index]
        accY_segment = accY[start_index:end_index]
        accZ_segment = accZ[start_index:end_index]
        time_segment = datetime_data[start_index:end_index]

        # Append to lists
        accX_segments.append(accX_segment)
        accY_segments.append(accY_segment)
        accZ_segments.append(accZ_segment)

        # Plot accX, accY, and accZ around the breath event
        plt.plot(time_segment, accX_segment, label='accX', color='blue', alpha=0.5)
        plt.plot(time_segment, accY_segment, label='accY', color='green', alpha=0.5)
        plt.plot(time_segment, accZ_segment, label='accZ', color='red', alpha=0.5)

        # Highlight the breath event
        plt.axvline(datetime_data[closest_time_index], color='black', linestyle='--', alpha=0.7)

    plt.xlabel('Datetime')
    plt.ylabel('Acceleration (g)')
    plt.title('Accelerometer Data (accX, accY, accZ) During Exhalation Breaths')
    plt.legend(loc='upper right')
    plt.xticks(rotation=45)
    plt.grid(True)
    plt.show()

    # Calculate the mean vector abar0
    mean_accX = np.mean(np.concatenate(accX_segments))
    mean_accY = np.mean(np.concatenate(accY_segments))
    mean_accZ = np.mean(np.concatenate(accZ_segments))
    abar0 = np.array([mean_accX, mean_accY, mean_accZ])

    return abar0

# Example usage (doesn't work currently)
#abar0 = plot_acc_for_exhalation_breaths(data_pkl)
#print("Average acceleration vector (abar0):", abar0)


In [ ]:
import numpy as np
import math

abar0 = [0, 0, -9.8] # over-writing to check 

def orientation_and_heading_correction(abar0, acc_data, mag_data):
    # Normalize abar0 to create abar
    abar = abar0 / np.linalg.norm(abar0)
    
    # Calculate initial pitch (p0) and roll (r0)
    p0 = np.arcsin(abar[0])
    r0 = np.arctan2(abar[1], abar[2])
    # Constrain p to [-pi / 2, pi / 2]
    if p0 > np.pi / 2:
        p0 = np.pi / 2 - p0
        r0 = r0 + np.pi

    print("p0:", p0)
    print("r0:", r0)
    
    # Define rotation matrices for pitch and roll
    def rotP(p):
        return np.array([[np.cos(p), 0, np.sin(p)],
                         [0, 1, 0],
                         [-np.sin(p), 0, np.cos(p)]])
    
    def rotR(r):
        return np.array([[1, 0, 0],
                         [0, np.cos(r), -np.sin(r)],
                         [0, np.sin(r), np.cos(r)]])
    
    # Calculate rotation matrix W
    W = np.matmul(rotP(p0), rotR(r0)).T
    print("W:", W)

    # Sanity check 1
    print("Sanity check 1: tag acc (abar) should equal product [0 0 -g]QW^-1 (here Q=I)")
    print("abar:", abar)
    print("Expected [0 0 -g]QW^-1:", np.matmul(np.array([0, 0, -1]), W.T))

    # Sanity check 2
    print("Sanity check 2: a_animal [0 0 -g] = abar * W abar times W should yield [0 0 -1]")
    print("abar * W:", np.matmul(abar, W))
    
    # Correct the accelerometer and magnetometer data for the entire dataset
    acc_corr = np.matmul(acc_data, W)
    mag_corr = np.matmul(mag_data, W)
    
    # Calculate magnitude of the corrected accelerometer vectors
    A = np.linalg.norm(acc_corr, axis=1)
    
    # Calculate pitch and roll in degrees from corrected accelerometer data
    pitch_deg = np.degrees(np.arcsin(acc_corr[:, 0] / A))
    roll_deg = np.degrees(np.arctan2(acc_corr[:, 1], acc_corr[:, 2]))
    
    b = data_pkl.data['CC-96']['field_intensity_mag2']
    i = data_pkl.data['CC-96']['inclination_angle2']

    

    # Calculate heading in degrees from corrected magnetometer data
    # TODO: flatten first
    heading_deg = np.degrees(np.arctan2(-mag_corr[:, 1], mag_corr[:, 0]))
    
    # Return the corrected pitch, roll, and heading for the entire dataset
    return pitch_deg, roll_deg, heading_deg, acc_corr, mag_corr

# Use the function to get corrected orientation and heading for the entire dataset
pitch_deg, roll_deg, heading_deg, acc_corr, mag_corr = orientation_and_heading_correction(abar0, acc_data, mag_data)


In [ ]:
?plot_tag_data_interactive

In [ ]:
# Save corrected accelerometer and magnetometer data back to data_pkl
data_pkl.data['CC-96']['corr_accX'] = acc_corr[:, 0]
data_pkl.data['CC-96']['corr_accY'] = acc_corr[:, 1]
data_pkl.data['CC-96']['corr_accZ'] = acc_corr[:, 2]
data_pkl.data['CC-96']['corr_magX'] = mag_corr[:, 0]
data_pkl.data['CC-96']['corr_magY'] = mag_corr[:, 1]
data_pkl.data['CC-96']['corr_magZ'] = mag_corr[:, 2]
# Add the calculated pitch, roll, and heading to the data_pkl structure
data_pkl.data['CC-96']['pitch_deg'] = pitch_deg
data_pkl.data['CC-96']['roll_deg'] = roll_deg
data_pkl.data['CC-96']['heading_deg'] = heading_deg


imu_channels_to_plot = ['depth', 'accX', 'accY', 'accZ', 'corr_accX', 'corr_accY', 'corr_accZ', 'magX', 'magY', 'magZ', 'pitch_deg', 'roll_deg', 'heading_deg']
ephys_channels_to_plot = []
imu_logger_to_use = 'CC-96'
ephys_logger_to_use = 'UF-01'

# Get the overlapping time range
imu_df = data_pkl.data[imu_logger_to_use]
ephys_df = data_pkl.data[ephys_logger_to_use]
start_time = max(imu_df['datetime'].min(), ephys_df['datetime'].min()).to_pydatetime()
end_time = min(imu_df['datetime'].max(), ephys_df['datetime'].max()).to_pydatetime()

# Define notes to plot
notes_to_plot = {
    'exhalation_breath': 'depth'
}

plot_tag_data_interactive(data_pkl, imu_channels_to_plot, imu_sampling_rate=1, ephys_channels=ephys_channels_to_plot, 
                          imu_logger=imu_logger_to_use, ephys_logger=ephys_logger_to_use, note_annotations= notes_to_plot,
                          time_range=(start_time, end_time), color_mapping_path=color_mapping_path)

CorrAccX: (AccX/10)*0.999954056289382 + (AccY/10)*(0.00561014920265745) + (AccZ/10)*(-0.00777248585299344)
CorrAccY: (AccX/10)*(0) + (AccY/10)*(0.810843233429072) + (AccZ/10)*(0.585263402924091)
CorrAccZ: (AccX/10)*0.0095856825740821 + (AccY/10)*(-0.585236513751672) + (AccZ/10)*(0.810805980282199)

Pitch: RadtoDeg(-ASin(CorrAccX/Sqrt(CorrAccX^2+CorrAccY^2+CorrAccZ^2)))
Roll: RadToDeg(ATan2((-CorrAccX/Sqrt(CorrAccY^2+CorrAccY^2+CorrAccZ^2)),(-CorrAccZ/Sqrt(CorrAccX^2+CorrAccY^2+CorrAccZ^2))))

In [ ]:
import numpy as np
from scipy.signal import butter, filtfilt

# Function to apply a low-pass filter to extract the static component (gravity)
def low_pass_filter(data, cutoff, fs, order=4):
    nyquist = 0.5 * fs
    normal_cutoff = cutoff / nyquist
    b, a = butter(order, normal_cutoff, btype='low', analog=False)
    filtered_data = filtfilt(b, a, data)
    return filtered_data

# Function to calculate ODBA
def calculate_odba(accX, accY, accZ, cutoff=0.1, fs=10):
    # Apply low-pass filter to get the static acceleration
    accX_static = low_pass_filter(accX, cutoff, fs)
    accY_static = low_pass_filter(accY, cutoff, fs)
    accZ_static = low_pass_filter(accZ, cutoff, fs)

    # Subtract the static component to get the dynamic acceleration
    accX_dynamic = accX - accX_static
    accY_dynamic = accY - accY_static
    accZ_dynamic = accZ - accZ_static

    # Calculate ODBA
    odba = np.abs(accX_dynamic) + np.abs(accY_dynamic) + np.abs(accZ_dynamic)
    
    return odba

# Example usage with your data
accX = data_pkl.data['CC-96']['accX_adjusted'].values
accY = data_pkl.data['CC-96']['accY_adjusted'].values
accZ = data_pkl.data['CC-96']['accZ_adjusted'].values

odba = calculate_odba(accX, accY, accZ)

data_pkl.data['CC-96']['odba'] = odba

imu_channels_to_plot = ['depth', 'accX', 'accY', 'accZ', 'odba', 'pitch_deg', 'roll_deg', 'heading_deg']
ephys_channels_to_plot = []
imu_logger_to_use = 'CC-96'
ephys_logger_to_use = 'UF-01'

# Get the overlapping time range
imu_df = data_pkl.data[imu_logger_to_use]
ephys_df = data_pkl.data[ephys_logger_to_use]
start_time = max(imu_df['datetime'].min(), ephys_df['datetime'].min()).to_pydatetime()
end_time = min(imu_df['datetime'].max(), ephys_df['datetime'].max()).to_pydatetime()

# Define notes to plot
notes_to_plot = {
    'exhalation_breath': 'depth'
}

plot_tag_data_interactive(data_pkl, imu_channels_to_plot, imu_sampling_rate=1, ephys_channels=ephys_channels_to_plot, 
                          imu_logger=imu_logger_to_use, ephys_logger=ephys_logger_to_use, note_annotations= notes_to_plot,
                          time_range=(start_time, end_time), color_mapping_path=color_mapping_path)

In [ ]:
import os
import trimesh
import numpy as np
from pythreejs import *
from IPython.display import display

# Define the path to the OBJ file
model_path = os.path.join(data_dir, '6_killerWhale_v017_LP.obj')

# Load the OBJ file using Trimesh
mesh = trimesh.load(model_path)

# Extract vertices and faces from the mesh
vertices = mesh.vertices
faces = mesh.faces.astype(np.uint32)  # Ensure the indices are unsigned integers

# Create BufferGeometry for PyThreeJS
geometry = BufferGeometry(
    attributes={
        'position': BufferAttribute(vertices, normalized=False),
        'index': BufferAttribute(faces.flatten(), normalized=False)
    }
)

# Create a material
material = MeshLambertMaterial(color='red') # , roughness=0.9, metalness=0.2
key_light = DirectionalLight(color='white', position=[3, 5, 1], intensity=0.75)

# Create the Mesh
mesh = Mesh(geometry=geometry, material=material, position=[0, 0, 200], rotation= [0, math.pi, 0, 'XYZ'])

# Move (translate) the mesh
mesh.position = [0, 0, 200]  # Move the mesh up by 1 unit on the Y-axis

# Rotate the mesh
#mesh.rotation = [0.5, 0.5, 0, 0]  # Rotate the mesh around the X and Y axes (in radians)

# Scale the mesh
mesh.scale = [1.5, 1.5, 1.5]  # Scale the mesh uniformly by 1.5 on all axes

# Create a scene, camera, and renderer
scene = Scene(children=[mesh, AmbientLight(color='#ffffff', intensity=0.5)])
camera = PerspectiveCamera(position=[0, 0, 5], up=[0, 1, 0], children=[key_light])
renderer = Renderer(camera=camera, scene=scene, controls=[OrbitControls(controlling=camera)], width=400, height=400)

# Display the model in the notebook
display(renderer)

ball = Mesh(geometry=SphereGeometry(), 
            material=MeshLambertMaterial(color='red'))


c = PerspectiveCamera(position=[0, 5, 5], up=[0, 1, 0], children=[key_light])

scene = Scene(children=[ball, c, AmbientLight(color='#777777')], background=None)

renderer = Renderer(camera=c, 
                    scene=scene,
                    alpha=True,
                    clearOpacity=0,
                    controls=[OrbitControls(controlling=c)])
display(renderer)

In [ ]:
from pyologger.process_data.feature_generation_utils import get_heart_rate
hr = get_heart_rate(data_pkl.data['UF-01']['ecg'])
data_pkl.data['UF-01']['hr'] = hr
data_pkl.info['UF-01']['channelinfo']['hr'] = 'Heart Rate'

print(hr)

from plotnine import ggplot, aes, geom_histogram, labs

# Assuming 'hr' is a pandas Series or a column in a DataFrame
# For demonstration, let's assume it's part of a DataFrame named df
df = pd.DataFrame({'hr': hr})

# Create the histogram
histogram_plot = (
    ggplot(df, aes(x='hr')) +
    geom_histogram(binwidth=5, fill='lightblue', color='white', alpha=0.7) +
    labs(title='Histogram of Heart Rate (HR)', x='Heart Rate (HR)', y='Count')
)

# To display the plot
print(histogram_plot)

imu_channels_to_plot = ['depth', 'accX', 'accY', 'accZ', 'odba']
ephys_channels_to_plot = ['ecg', 'hr']
imu_logger_to_use = 'CC-96'
ephys_logger_to_use = 'UF-01'

# Get the overlapping time range
imu_df = data_pkl.data[imu_logger_to_use]
ephys_df = data_pkl.data[ephys_logger_to_use]
start_time = max(imu_df['datetime'].min(), ephys_df['datetime'].min()).to_pydatetime()
end_time = min(imu_df['datetime'].max(), ephys_df['datetime'].max()).to_pydatetime()

# Define notes to plot
notes_to_plot = {
    'exhalation_breath': 'depth',
    'heartbeat_manual_ok': 'ecg',
}

plot_tag_data_interactive(data_pkl, imu_channels_to_plot, imu_sampling_rate=1, ephys_channels=ephys_channels_to_plot, 
                          imu_logger=imu_logger_to_use, ephys_logger=ephys_logger_to_use, note_annotations= notes_to_plot,
                          time_range=(start_time, end_time), color_mapping_path=color_mapping_path)